In [1]:
import os
print(f"Current working directory: {os.getcwd()}")
os.chdir("../")

Current working directory: /home/vairavan/projects/Algotrader/notebooks


In [2]:
import logging
from src.utils.logging_setup import setup_general_logging

setup_general_logging(
    "/home/vairavan/projects/Algotrader/logs/",
    level=logging.INFO,
    log_to_console=True,
    log_to_file=False
)
# Verify logging is working
logging.info("Logging setup complete")

2025-05-27 00:01:30 -0500 | INFO     | /home/vairavan/projects/Algotrader/src/utils/logging_setup.py | General logging configured successfully.
2025-05-27 00:01:30 -0500 | INFO     | root | Logging setup complete


In [3]:
from ib_async import util
util.startLoop()  # Start the asyncio event loop
from src.utils.helpers import load_json_config
ibkr_config = load_json_config("config/accounts/ibkr_default.json")

from src.data_management.ibkr_helper import IBKRHelper
timezone = ibkr_config["ibkr"]["trading_timezone"]
ibkr = IBKRHelper(market_timezone_str=timezone)
await ibkr.connect(port=IBKRHelper.TWS_PORT,force_new=True,client_id=36)  # Connect to TWS or IB Gateway

2025-05-27 00:01:36 -0500 | INFO     | src.data_management.ibkr_helper | Successfully connected to IB at 127.0.0.1:7497


True

2025-05-27 00:02:49 -0500 | INFO     | src.data_management.ibkr_helper | IB Info [2108]: Market data farm connection is inactive but should be available upon demand.uscrypto. reqId: -1. Live subscriptions may be delayed or closed.
2025-05-27 00:02:50 -0500 | INFO     | src.data_management.ibkr_helper | IB Info [2108]: Market data farm connection is inactive but should be available upon demand.uscrypto. reqId: -1. Live subscriptions may be delayed or closed.
2025-05-27 00:02:50 -0500 | INFO     | src.data_management.ibkr_helper | IB Info [2108]: Market data farm connection is inactive but should be available upon demand.jfarm. reqId: -1. Live subscriptions may be delayed or closed.
2025-05-27 00:02:50 -0500 | INFO     | src.data_management.ibkr_helper | IB Info [2108]: Market data farm connection is inactive but should be available upon demand.jfarm. reqId: -1. Live subscriptions may be delayed or closed.
2025-05-27 00:02:50 -0500 | INFO     | src.data_management.ibkr_helper | IB Info [

In [11]:
import datetime
from ib_async import Index, Contract
from rich.progress import track
from tqdm import tqdm
DATA_DIR = "/home/vairavan/projects/Algotrader/data/historical/NIFTY/"
strike_price_range = 500  # Range around the index price to filter contracts

today = datetime.datetime.now(tz=ibkr.market_timezone).replace(hour=0,minute=0,second=0,microsecond=0)
days_since_thursday = (today.weekday() - 3) % 7 
last_week_thursday = today - datetime.timedelta(days=days_since_thursday)
this_week_thursday = last_week_thursday + datetime.timedelta(days=7)

all_weekdays = [last_week_thursday + datetime.timedelta(days=i) 
                for i in range(0,(today - last_week_thursday).days+1) 
                if (last_week_thursday + datetime.timedelta(days=i)).weekday() <5]

all_contracts = await ibkr.get_option_chains_new(symbol="NIFTY50",expiry_weeks=1)
logging.info(f"Found {len(all_contracts)} contracts for NIFTY options")
all_contracts = [c for c in all_contracts if c.lastTradeDateOrContractMonth == this_week_thursday.strftime("%Y%m%d")]
logging.info(f"Filtered to {len(all_contracts)} contracts for today's date: {this_week_thursday.strftime('%Y-%m-%d')}")

idx = Index(symbol="NIFTY50", exchange="NSE", currency="INR")
index_history = await ibkr.get_historical_bars(contract=idx, start_date=all_weekdays[0], end_date=all_weekdays[-1], bar_size="1 day", return_bars=True)
logging.info(f"Retrieved {len(index_history)} days of index data for NIFTY50")

for i, bar in enumerate(index_history):

    day = bar.date
    contracts = [c for c in all_contracts if c.strike > index_history[i].close-strike_price_range and c.strike < index_history[i].close+strike_price_range]
    for c in tqdm(contracts, total=len(contracts), desc=f"{i+1}/{len(index_history)}: Processing contracts for {day.date()}", ncols=30):
        data_logger = ibkr.get_bardata_logger(contract=c, bar_size="1 day", data_dir=DATA_DIR)
        hist_bars = await ibkr.get_historical_bars(c, day, day, bar_size="1 day", return_bars=True)
        
        if hist_bars is not None and len(hist_bars) > 0:
            data_logger.log_bars(hist_bars)
            data_logger = ibkr.get_bardata_logger(contract=c, bar_size="5 secs", data_dir=DATA_DIR)
            hist_bars = await ibkr.get_historical_bars(c, day, day, bar_size="5 secs", return_bars=True )
            if hist_bars is not None and len(hist_bars) > 0:
                data_logger.log_bars(hist_bars)


2025-05-27 00:10:38 -0500 | INFO     | src.data_management.ibkr_helper | Retrieved 540 from option chains for symbol=NIFTY50, strike_range= +/- None,expiry month = None and expiry week = 1.
2025-05-27 00:10:38 -0500 | INFO     | root | Found 540 contracts for NIFTY options
2025-05-27 00:10:38 -0500 | INFO     | root | Filtered to 540 contracts for today's date: 2025-05-29
2025-05-27 00:10:39 -0500 | INFO     | root | Retrieved 3 days of index data for NIFTY50


1/3: Processing contracts for 
2/3: Processing contracts for 
3/3: Processing contracts for 


In [10]:
today = datetime.datetime.now(tz=ibkr.market_timezone).replace(hour=0,minute=0,second=0,microsecond=0)
days_since_thursday = (today.weekday() - 3) % 7 
last_week_thursday = today - datetime.timedelta(days=days_since_thursday)

[last_week_thursday + datetime.timedelta(days=i) 
                for i in range(0,(today - last_week_thursday).days+1) 
                if (last_week_thursday + datetime.timedelta(days=i)).weekday() <5]
this_week_thursday = last_week_thursday + datetime.timedelta(days=7)
this_week_thursday

datetime.datetime(2025, 5, 29, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='Asia/Calcutta'))

In [2]:
import pandas as pd 
df = pd.read_csv("/home/vairavan/projects/Algotrader/data/historical/NIFTY/nifty50_daily.csv")
df 

,date,open,high,low,close,volume,average,barCount
0,2025-04-01 00:00:00+05:30,23353.00,23563.20,23136.40,23165.70,0.0,0.0,43377
1,2025-04-02 00:00:00+05:30,23209.10,23349.90,23160.40,23332.40,0.0,0.0,42908
2,2025-04-03 00:00:00+05:30,23160.40,23306.30,23159.60,23250.10,0.0,0.0,43335
3,2025-04-04 00:00:00+05:30,23208.20,23210.50,22858.10,22904.50,0.0,0.0,43654
4,2025-04-07 00:00:00+05:30,21807.80,22253.20,21796.90,22161.60,0.0,0.0,43992
5,2025-04-08 00:00:00+05:30,22473.40,22696.70,22272.90,22535.90,0.0,0.0,43792
6,2025-04-09 00:00:00+05:30,22453.30,22468.30,22353.50,22399.20,0.0,0.0,43414
7,2025-04-11 00:00:00+05:30,22732.50,22923.90,22709.90,22828.60,0.0,0.0,43100
8,2025-04-15 00:00:00+05:30,23346.80,23357.10,23211.40,23328.55,0.0,0.0,43335
9,2025-04-16 00:00:00+05:30,23345.25,23452.05,23274.00,23437.20,0.0,0.0,43601
